In [1]:
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [10]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
47/47 [==============================] - 1s 20ms/step - loss: 2.2418 - accuracy: 0.1277
Epoch 2/200
47/47 [==============================] - 0s 1ms/step - loss: 2.1656 - accuracy: 0.2340
Epoch 3/200
47/47 [==============================] - 0s 2ms/step - loss: 2.0875 - accuracy: 0.2553
Epoch 4/200
47/47 [==============================] - 0s 872us/step - loss: 2.0151 - accuracy: 0.2553
Epoch 5/200
47/47 [==============================] - 0s 1ms/step - loss: 1.8735 - accuracy: 0.4043
Epoch 6/200
47/47 [==============================] - 0s 2ms/step - loss: 1.8162 - accuracy: 0.3617
Epoch 7/200
47/47 [==============================] - 0s 1ms/step - loss: 1.6325 - accuracy: 0.5957
Epoch 8/200
47/47 [==============================] - 0s 1ms/step - loss: 1.4418 - accuracy: 0.5532
Epoch 9/200
47/47 [==============================] - 0s 1ms/step - loss: 1.3227 - accuracy: 0.6170
Epoch 10/200
47/47 [==============================] - 0s 1ms/step - loss: 1.1954 - accuracy: 0.6170
Epoch 

47/47 [==============================] - 0s 2ms/step - loss: 0.0505 - accuracy: 0.9787
Epoch 162/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0337 - accuracy: 0.9787
Epoch 163/200
47/47 [==============================] - 0s 851us/step - loss: 0.0443 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 787us/step - loss: 0.0337 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 745us/step - loss: 0.0180 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 745us/step - loss: 0.0285 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 723us/step - loss: 0.0252 - accuracy: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 958us/step - loss: 0.0142 - accuracy: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 894us/step - loss: 0.0293 - accuracy: 0.9787
Epoch 170/200
47/47 [==============================] - 0s 723us/step - loss: 0.0175 - accur